# Holdout Validation

You're probably used to 80-20 train/test data splitting. But, here, we're going to do 50-50 train-test one way and then train-test using the complementary data.. So we'll end up with two models and two error numbers which we'll then average. 

The ultimate goal DQ hasn't told me yet :)

In [21]:
import pandas as pd
from pandas.api.types import is_numeric_dtype
import numpy as np

In [4]:
dc_listings = pd.read_csv("tomslee_airbnb_washington_1433_2017-07-11.csv")
dc_listings.head()

,room_id,survey_id,host_id,room_type,country,city,borough,neighborhood,reviews,overall_satisfaction,accommodates,bedrooms,bathrooms,price,minstay,last_modified,latitude,longitude,location
0,3732219,1433,280636,Shared room,NaN,Washington,NaN,Columbia Heights,0,0.0,3,1.0,NaN,129.0,NaN,2017-07-11 08:53:56.381540,38.931081,-77.030618,0101000020E6100000D02A33A5F54153C0A77686A92D77...
1,15087225,1433,90860645,Shared room,NaN,Washington,NaN,Brentwood,6,5.0,4,1.0,NaN,118.0,NaN,2017-07-11 08:53:55.616987,38.908054,-77.003306,0101000020E61000005B785E2A364053C041800C1D3B74...
2,19634784,1433,138150306,Shared room,NaN,Washington,NaN,South West,1,0.0,4,1.0,NaN,84.0,NaN,2017-07-11 08:53:53.434225,38.884121,-77.019518,0101000020E6100000BCEB6CC83F4153C0795A7EE02A71...
3,18547685,1433,26180779,Shared room,NaN,Washington,NaN,Shaw,11,5.0,2,1.0,NaN,74.0,NaN,2017-07-11 08:53:49.654605,38.910593,-77.023461,0101000020E6100000D0EE9062804153C0B77BB94F8E74...
4,13878076,1433,2387207,Shared room,NaN,Washington,NaN,Cleveland Park,2,0.0,2,1.0,NaN,50.0,NaN,2017-07-11 08:53:48.721169,38.935485,-77.059807,0101000020E61000009A44BDE0D34353C00473F4F8BD77...


Putting this one here (though it's not necessary) just to show one way of cleaning a should-be-numeric column :

In [17]:
is_numeric_dtype( dc_listings['city'].dtype )

False

In [19]:
if not is_numeric_dtype( dc_listings['price'] ) :
    dc_listings['price'] = dc_listings['price'].str.replace('[ $,]','')  # DQ doesn't know yet that str.replace takes a regex
    try :
        dc_listings['price'] = dc_listings['price'].astype('float')
    except ValueError as err :
        print("Price needs pre-processing to remove bad chars : {}".format(err) )

Remove any bias in the data by randomizing (shuffling)

In [23]:
dc_listings = dc_listings.loc[ np.random.permutation( len(dc_listings) ) ]

In [24]:
N_split = int( len( dc_listings )/2 )
train_df = dc_listings[ : N_split ]
test_df  = dc_listings[ N_split : ]

In [26]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [28]:
knn = KNeighborsRegressor( )
knn.fit( train_df[["accommodates"], train_df['price'])
predictions = knn.predict( test_df[['accommodates']])
rmse1 = mean_squared_error( test_df['price'], predictions )**0.5
knn.fit( test_df[['accommodates'] , test_df['price'])
predictions = knn.predict( train_df[['accommodates']])
rmse2 = mean_squared_error( train_df['price'], predictions )**0.5
avg_rmse = np.mean([rmse1, rmse2])
avg_rmse

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (<ipython-input-28-035685c0dc67>, line 2)